# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116886e+02     1.497226e+00
 * time: 0.0959329605102539
     1     1.061801e+01     8.705710e-01
 * time: 1.0360329151153564
     2    -1.175583e+01     9.920153e-01
 * time: 1.1592960357666016
     3    -3.431268e+01     7.513608e-01
 * time: 1.3359110355377197
     4    -4.773551e+01     5.180473e-01
 * time: 1.4955799579620361
     5    -5.695168e+01     2.193029e-01
 * time: 1.655040979385376
     6    -5.980109e+01     1.041303e-01
 * time: 1.7759740352630615
     7    -6.085371e+01     4.902186e-02
 * time: 1.8931388854980469
     8    -6.124303e+01     6.633842e-02
 * time: 2.015105962753296
     9    -6.155696e+01     3.404328e-02
 * time: 2.1303958892822266
    10    -6.177759e+01     2.664072e-02
 * time: 2.256103038787842
    11    -6.194409e+01     2.485251e-02
 * time: 2.3813679218292236
    12    -6.201045e+01     2.108280e-02
 * time: 2.4990880489349365
    13    -6.208228e+01     1.674172e-02
 * time: 2.6264739036560

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671049
    AtomicLocal         -18.8557633
    AtomicNonlocal      14.8522632
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485352 
    Xc                  -19.3336813

    total               -62.261666452225
